# Movie Maps
---
Webscrapper for Movie data.
---
### Dependencies

In [1]:
from bs4 import BeautifulSoup
import requests
from splinter import Browser
from os.path import basename
from pprint import pprint
import time
from datetime import datetime as dt
import pandas as pd
import numpy as np

In [2]:
# Splinter set up
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

---
### Extraction
- Scrape data off of Box Office Mojo

In [3]:
def scrapeMovies(id_url, data_url):
    browser.visit(id_url)
    # Direct browser to page
    html = browser.html
    bom_soup = BeautifulSoup(html, 'html.parser')

    # Focus on 'body' content
    main_body = bom_soup.find('div', id='body')

    # Focus on the 4th table in body
    main_table = main_body.findAll('table')[3]
    
    # Extract the rows associated with movie links
    white_rows = main_table.findAll("tr", bgcolor="#ffffff")
    bluey_rows = main_table.findAll("tr", bgcolor="#f4f4ff")

    # Extract movie IDs from the links
    movie_ids = []
    movie_ttl = []

    for row in white_rows:
        link = row.find('a')
        if (link):
            addr = link['href'].split('=')[1].split('.')[0]
            movie_ids.append(addr)
            movie_ttl.append(link.get_text())
    for row in bluey_rows:
        link = row.find('a')
        if (link):
            addr = link['href'].split('=')[1].split('.')[0]
            movie_ids.append(addr)
            movie_ttl.append(link.get_text())

    # Scrape the weekly gross data on the targeted movies
    movie_pds = []
    counter = 0
    for ids in movie_ids:
        time.sleep(1)
        print(f"{counter}: {ids};")
        wk_html = pd.read_html(data_url + ids + '.htm')
        if (wk_html):
            # slice column
#             wk_col = wk_html[0].iloc[10, :9]

            # slice data
            wk_pd = wk_html[0].iloc[11:, :9]

#             # fix column labels
#             wk_col[6:9] = wk_col[5:8]
#             wk_col[0] = 'Date'
#             wk_col[4:6] = ['Theaters', 'Change']
            wk_col = ['Date','Rank','WeeklyGross','%Change',
                      'Theaters','Change','Avg.','Gross-to-Date',
                      'Week#']
            
            # set column labels
            wk_pd.columns = wk_col
            wk_pd.dropna(inplace=True)
            wk_pd.reset_index(inplace=True, drop=True)

            movie_pds.append(wk_pd)
        counter = counter + 1
    # Return findings
    return {"titles":movie_ttl, "ids":movie_ids, "pds":movie_pds}

In [4]:
def scrapeFranchiseMovies(id_url, data_url):
    browser.visit(id_url)
    # Direct browser to page
    html = browser.html
    bom_soup = BeautifulSoup(html, 'html.parser')

    # Focus on 'body' content
    main_body = bom_soup.find('div', id='body')

    # Focus on the 4th table in body
    main_table = main_body.findAll('table')[3]
    
    # Extract the rows associated with movie links
    white_rows = main_table.findAll("tr", bgcolor="#ffffff")
    bluey_rows = main_table.findAll("tr", bgcolor="#f4f4ff")

    # Extract movie IDs from the links
    movie_ids = []
    movie_ttl = []
    limit = 0
    for row in white_rows:
        link = row.find('a')
        if (link):
            addr = link['href'].split('=')[1].split('.')[0]
            movie_ids.append(addr)
            movie_ttl.append(link.get_text())
            limit = limit+1
            if (limit == 10): 
                break
    limit = 0
    for row in bluey_rows:
        link = row.find('a')
        if (link):
            addr = link['href'].split('=')[1].split('.')[0]
            movie_ids.append(addr)
            movie_ttl.append(link.get_text())
            limit = limit+1
            if (limit == 10): 
                break

    # Scrape the weekly gross data on the targeted movies
    movie_pds = []
    counter = 0
    for ids in movie_ids:
        time.sleep(1)
        print(f"{counter}: {ids};")
        wk_html = pd.read_html(data_url + ids + '.htm')
        if (wk_html):
#             # slice column
#             wk_col = wk_html[0].iloc[10, :9]

            # slice data
            wk_pd = wk_html[0].iloc[11:, :9]

            # fix column labels
#             wk_col[6:9] = wk_col[5:8]
#             wk_col[0] = 'Date'
#             wk_col[4:6] = ['Theaters', 'Change']
            wk_col = ['Date','Rank','WeeklyGross','%Change',
                      'Theaters','Change','Avg.','Gross-to-Date',
                      'Week#']
    
            # set column labels
            wk_pd.columns = wk_col
            wk_pd.dropna(inplace=True)
            wk_pd.reset_index(inplace=True, drop=True)

            movie_pds.append(wk_pd)
        counter = counter + 1
    # Return findings
    return {"titles":movie_ttl, "ids":movie_ids, "pds":movie_pds}

In [5]:
# # Direct browser to page
# html = browser.html
# bom_soup = BeautifulSoup(html, 'html.parser')

# # Focus on 'body' content
# main_body = bom_soup.find('div', id='body')

# # Focus on the 4th table in body
# main_table = main_body.findAll('table')[3]

# # Inspect html
# print(main_table)


In [6]:
# # Extract the rows associated with movie links
# white_rows = main_table.findAll("tr", bgcolor="#ffffff")
# bluey_rows = main_table.findAll("tr", bgcolor="#f4f4ff")

# # Examine row structure
# print(white_rows[0])

In [7]:
# # Extract movie IDs from the links
# movie_ids = []
# movie_ttl = []

# for row in white_rows:
#     link = row.find('a')
#     if (link):
#         addr = link['href'].split('=')[1].split('.')[0]
#         movie_ids.append(addr)
#         movie_ttl.append(link.get_text())
# for row in bluey_rows:
#     link = row.find('a')
#     if (link):
#         addr = link['href'].split('=')[1].split('.')[0]
#         movie_ids.append(addr)
#         movie_ttl.append(link.get_text())

# # Check extraction results
# print(len(movie_ids))
# for i in range(5):
#     print(f"{movie_ttl[i]}: {movie_ids[i]}")

In [8]:
# # Determine how to scrape movie data from webpage 
# wk_html = pd.read_html('https://www.boxofficemojo.com/movies/?page=weekly&id=marvel2017b.htm')

# # slice column
# wk_col = wk_html[0].iloc[10, 0:9]

# # slice data
# wk_pd = wk_html[0].iloc[11:, 0:9]

# # fix column labels
# wk_col[6:9] = wk_col[5:8]
# wk_col[0] = 'Date'
# wk_col[4:6] = ['Theaters', 'Change']

# # set column labels
# wk_pd.columns = wk_col

# # preview results
# wk_pd.head()

In [9]:
# # Base url for targeted scrape data
# wk_base_url = 'https://www.boxofficemojo.com/movies/?page=weekly&id='


In [10]:
# movie_pds = []
# counter = 0
# for ids in movie_ids:
# #     if((counter > 0) & (counter%20 == 0)):
#     time.sleep(1)
#     print(f"{counter}: {ids};")
#     wk_html = pd.read_html(wk_base_url + ids + '.htm')
#     if (wk_html):
#         # slice column
#         wk_col = wk_html[0].iloc[10, :9]

#         # slice data
#         wk_pd = wk_html[0].iloc[11:, :9]

#         # fix column labels
#         wk_col[6:9] = wk_col[5:8]
#         wk_col[0] = 'Date'
#         wk_col[4:6] = ['Theaters', 'Change']

#         # set column labels
#         wk_pd.columns = wk_col

#         movie_pds.append(wk_pd)
#     counter = counter + 1
    

# print(f'"{movie_ttl[1]}": {movie_ids[1]}')
# movie_pds[1].head()

#### Function Based Scraping

In [11]:
# Common base url
base_url = 'https://www.boxofficemojo.com' 

# Category URL's for scraping:
y2018_url = '/yearly/chart/?yr=2018&p=.htm'
y2017_url = '/yearly/chart/?yr=2017&p=.htm'
y2016_url = '/yearly/chart/?yr=2016&p=.htm'
marEU_url = '/franchises/chart/?id=avengers.htm'
jBond_url = '/franchises/chart/?id=jamesbond.htm'


# Data URL's for scraping:
wkly_url = '/movies/?page=weekly&id='

In [12]:
# res2018 = scrapeMovies(base_url+y2018_url, base_url+wkly_url)
# res2018['pds'][0]

In [13]:
res2017 = scrapeMovies(base_url+y2017_url, base_url+wkly_url)

0: starwars8;
1: wonderwoman;
2: marvel17a;
3: it;
4: despicableme3;
5: wolverine2017;
6: pixar1117;
7: blumhouse2;
8: bossbaby;
9: potc5;
10: cars3;
11: split2017;
12: transformers5;
13: fiftyshadesdarker;
14: pitchperfect3;
15: murderorientexpress17;
16: kingsman2;
17: johnwick2;
18: powerrangers16;
19: untitledstevenspielberg;
20: hitmansbodyguard;
21: captainunderpants;
22: adogspurpose;
23: ninjago;
24: theshack;
25: blumhousehorror2018;
26: thecoldestcity;
27: darktower;
28: amadeahalloween2;
29: greatwall;
30: goinginsty2017;
31: xxx3;
32: thebigsick;
33: thelamb;
34: kingarthur2016;
35: americanassassin;
36: everythingeverything;
37: geostorm;
38: fistfight;
39: kidnap2015;
40: mountainbetweenus;
41: itonya;
42: mollysgame;
43: rings;
44: homeagain;
45: thehouse;
46: gifted;
47: thebyebyeman;
48: rockthatbody;
49: leap;
50: beautyandthebeast2017;
51: jumanji2016;
52: spiderman2017;
53: marvel2017;
54: dcfilm1117;
55: furious8;
56: chrisnolan2017;
57: lego2;
58: greatestshowman;

In [14]:
res2017['pds'][0].head()

,Date,Rank,WeeklyGross,%Change,Theaters,Change,Avg.,Gross-to-Date,Week#
0,Dec 1521,1,"$296,602,356",-,4232,-,"$70,086","$296,602,356",1
1,Dec 2228,1,"$168,095,872",-43.3%,4232,-,"$39,720","$464,698,228",2
2,Dec 29Jan 4,2,"$84,264,374",-49.9%,4232,-,"$19,911","$548,962,602",3
3,Jan 511,3,"$31,311,982",-62.8%,4232,-,"$7,399","$580,274,584",4
4,Jan 1218,5,"$17,443,892",-44.3%,3090,-1142,"$5,645","$597,718,476",5


In [15]:
res2016 = scrapeMovies(base_url+y2016_url, base_url+wkly_url)

0: starwars2016;
1: marvel2016;
2: junglebook2015;
3: disney2016;
4: dc2016;
5: disney1116;
6: marvel716;
7: bourne5;
8: x-men2016;
9: lalaland;
10: ghostbusters2016;
11: tarzan2016;
12: untitledlucasmoore;
13: id42;
14: arrival2016;
15: sausageparty;
16: ridealong2;
17: peregrine;
18: tmnt2016;
19: alice2;
20: thegirlonthetrain2016;
21: storks;
22: newline0116;
23: allegiant;
24: iceage5;
25: londonhasfallen;
26: deepwaterhorizon;
27: mybigfatgreekwedding2;
28: fences;
29: bfg;
30: theshallows;
31: assassinscreed;
32: 13hoursthesecretsoldiersofbenghazi;
33: huntsman;
34: manchesterbythesea;
35: howtobesingle;
36: armsandthedudes;
37: moneymonster;
38: nerve;
39: niceguys;
40: dirtygrandpa;
41: 5thwave;
42: mothersday;
43: godsofegypt;
44: hailcaesar;
45: zoolander2;
46: finesthours;
47: hellorhighwater;
48: benhur2016;
49: bridgetjonessbaby;
50: pixar2015;
51: illumination2015;
52: deadpool2016;
53: superman2015;
54: illumination2016;
55: fantasticbeasts;
56: hiddenfigures;
57: startr

In [16]:
resMarCU = scrapeFranchiseMovies(base_url+marEU_url, base_url+wkly_url)

0: marvel2017b;
1: avengers11;
2: ironman3;
3: marvel17a;
4: marvel2014a;
5: marvel2017;
6: marvel14b;
7: ant-manandthewasp;
8: thor;
9: captainamerica;
10: marvel0518;
11: avengers2;
12: marvel2016;
13: spiderman2017;
14: ironman;
15: ironman2;
16: marvel716;
17: thor2;
18: antman;
19: incrediblehulk;


In [17]:
resMarCU['pds'][0].head()

,Date,Rank,WeeklyGross,%Change,Theaters,Change,Avg.,Gross-to-Date,Week#
0,Feb 1622,1,"$291,954,422",-,4020,-,"$72,625","$291,954,422",1
1,Feb 23Mar 1,1,"$143,445,615",-50.9%,4020,-,"$35,683","$435,400,037",2
2,Mar 28,1,"$85,479,564",-40.4%,4084,+64,"$20,930","$520,879,601",3
3,Mar 915,1,"$57,496,927",-32.7%,3942,-142,"$14,586","$578,376,528",4
4,Mar 1622,1,"$35,881,708",-37.6%,3834,-108,"$9,359","$614,258,236",5


---
### Transformation
- Convert PDs to a csv format for chord diagram

In [18]:
# # Determine how to reformat dates
# dates = res2018['pds'][0]['Date']
# dates
# for date in dates:
#     p_date = date.split('')[0].split(' ')
#     if (len(p_date[1])==1):
#         p_date[1] = '0'+ p_date[1]
#     start_str = "-".join(p_date)+'-18'
#     start_date = dt.strptime(start_str, '%b-%d-%y')
#     week_nm = start_date.strftime('%U')
# #     print(f"{start_str} = week # {week_nm}")

In [19]:
# function to convert weekend to calendar week number 
def getWkNum(datestring, yr):
    p_date = datestring.split('')[0].split(' ')
    if (len(p_date[1])==1):
        p_date[1] = '0'+ p_date[1]
    start_str = '-'.join(p_date) + f'-{yr}'
    start_date = dt.strptime(start_str, '%b-%d-%y')
    return start_date.strftime('%U')

getWkNum("Dec 30", 18)

'52'

In [20]:
# Function to convert gross strings into numbers
def stripUSD(amount):
    return ''.join(amount.replace('$', '').split(','))

In [21]:
# cal2018_cols = ['Year', 'BOM_id', 'Movie']
# cal_rows = []
# for i in np.arange(53):
#     lbl = i
#     if i < 10:
#         lbl = f'0{i}'
#     cal2018_cols.append(f'W_{lbl}')
# for i in np.arange(53):
#     lbl = i
#     if i < 10:
#         lbl = f'0{i}'
#     cal2018_cols.append(f'T_{lbl}')


In [22]:
# # for each movie in the list
# for i in np.arange(len(res2018['pds'])):
#     # get the title and id to start the row
#     new_row = ['2018', res2018['ids'][i], res2018['titles'][i]]
#     # add in 0's 
#     for j in np.arange(106):
#         new_row.append('0')

#     itr_pds = res2018['pds'][i]
#     last_col = 56
#     last_grs = 0
#     for ind, row in itr_pds.iterrows():
#         calWkNum = getWkNum(row['Date'], 18)
# #         print(f"row {i}: {calWkNum} > {int(calWkNum)+3} = {row['WeeklyGross']} > {int(calWkNum)+56} = {row['Gross-to-Date']}")
#         last_col = 56 + int(calWkNum)
#         last_grs = stripUSD(row['Gross-to-Date'])
#         new_row[3 + int(calWkNum)] = stripUSD(row['WeeklyGross'])
#         new_row[last_col] = last_grs
#     for j in np.arange(last_col+1, 109):
#         new_row[j] = last_grs
#     cal_rows.append(new_row)

In [23]:
# m2018cal_df = pd.DataFrame(cal_rows, columns=cal2018_cols)
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#     display(m2018cal_df)

In [24]:
# m2018cal_df.to_csv('Data/caly_mov_gross_2018.csv', index=False)

In [25]:
# # Create column labels
# wkl2018_cols = ['Year', 'BOM_id', 'Movie']
# wkl_rows = []

# for i in np.arange(52):
#     lbl = i+1
#     if lbl < 10:
#         lbl = f'0{lbl}'
#     wkl2018_cols.append(f'W_{lbl}')
# for i in np.arange(52):
#     lbl = i+1
#     if lbl < 10:
#         lbl = f'0{lbl}'
#     wkl2018_cols.append(f'T_{lbl}')

# # Look for the longest movie duration
# max_dur = 0

# # Make a row for each movie in the list
# for i in np.arange(len(res2018['pds'])):
#     # get the title and id to start the row
#     new_row = ['2018', res2018['ids'][i], res2018['titles'][i]]
#     # add in 0's 
#     for j in np.arange(104):
#         new_row.append('0')

#     itr_pds = res2018['pds'][i]
#     last_col = 56
#     last_grs = 0
#     row_count = itr_pds.shape[0]
#     if (row_count > max_dur):
#         max_dur = row_count
    
#     for ind, row in itr_pds.iterrows():
#         #wkNum = row['Week#']
#         last_col = 55 + ind
#         last_grs = stripUSD(row['Gross-to-Date'])
#         new_row[3 + ind] = stripUSD(row['WeeklyGross'])
#         new_row[last_col] = last_grs
#     for j in np.arange(last_col+1, 107):
#         new_row[j] = last_grs
#     wkl_rows.append(new_row)

# # Check the longest running movie
# print(max_dur)

# # Create dataframe out of columns and rows
# m2018wkl_df = pd.DataFrame(wkl_rows, columns=wkl2018_cols)
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#     display(m2018wkl_df)

In [26]:
# m2018wkl_df.to_csv('Data/wkly_mov_gross_2018.csv', index=False)

#### Function Based Transformation:

In [27]:
def transform_movie(scrape_res, affix, year):
#     cal_cols = ['Year', 'BOM_id', 'Movie']
#     cal_rows = []
    
#     for i in np.arange(53):
#         lbl = i
#         if i < 10:
#             lbl = f'0{i}'
#         cal_cols.append(f'W_{lbl}')
    
#     for i in np.arange(53):
#         lbl = i
#         if i < 10:
#             lbl = f'0{i}'
#         cal_cols.append(f'T_{lbl}')
        
#     # for each movie in the list
#     for i in np.arange(len(scrape_res['pds'])):
#         # get the title and id to start the row
#         new_row = [year, scrape_res['ids'][i], scrape_res['titles'][i]]
#         # add in 0's 
#         for j in np.arange(106):
#             new_row.append('0')

#         itr_pds = scrape_res['pds'][i]
#         last_col = 56
#         last_grs = 0
#         for ind, row in itr_pds.iterrows():
#             calWkNum = getWkNum(row['Date'], year)
#             last_col = 56 + int(calWkNum)
#             last_grs = stripUSD(row['Gross-to-Date'])
#             new_row[3 + int(calWkNum)] = stripUSD(row['WeeklyGross'])
#             new_row[last_col] = last_grs
#         for j in np.arange(last_col+1, 109):
#             new_row[j] = last_grs
#         cal_rows.append(new_row)
    
#     mov_cal_df = pd.DataFrame(cal_rows, columns=cal_cols)
#     save_path = f'Data/caly_mov_gross_{affix}.csv'
#     mov_cal_df.to_csv(save_path, index=False)
    
    # Create column labels
    wkl_cols = ['Year', 'BOM_id', 'Movie']
    wkl_rows = []

    for i in np.arange(52):
        lbl = i+1
        if lbl < 10:
            lbl = f'0{lbl}'
        wkl_cols.append(f'W_{lbl}')
    for i in np.arange(52):
        lbl = i+1
        if lbl < 10:
            lbl = f'0{lbl}'
        wkl_cols.append(f'T_{lbl}')

    # Look for the longest movie duration
    max_dur = 0

    # Make a row for each movie in the list
    for i in np.arange(len(scrape_res['pds'])):
        # get the title and id to start the row
        new_row = [year, scrape_res['ids'][i], scrape_res['titles'][i]]
        # add in 0's 
        for j in np.arange(104):
            new_row.append('0')

        itr_pds = scrape_res['pds'][i]
        last_col = 56
        last_grs = 0
        row_count = itr_pds.shape[0]
        if (row_count > max_dur):
            max_dur = row_count

        for ind, row in itr_pds.iterrows():
            #wkNum = row['Week#']
            last_col = 55 + ind
            try:
                last_grs = stripUSD(row['Gross-to-Date'])
            except:
                print(f'{i}: {scrape_res["titles"][i]} >>')
                print(row)
            new_row[3 + ind] = stripUSD(row['WeeklyGross'])
            new_row[last_col] = last_grs
        for j in np.arange(last_col+1, 107):
            new_row[j] = last_grs
        wkl_rows.append(new_row)
        
    mov_wkl_df = pd.DataFrame(wkl_rows, columns=wkl_cols)
    save_path = f'Data/wkly_mov_gross_{affix}.csv'
    mov_wkl_df.to_csv(save_path, index=False)

In [28]:
res2017['pds'][33]

,Date,Rank,WeeklyGross,%Change,Theaters,Change,Avg.,Gross-to-Date,Week#
0,Nov 1723,7,"$15,157,196",-,2837,-,"$5,343","$15,157,196",1
1,Nov 2430,7,"$8,122,457",-46.4%,2837,-,"$2,863","$23,279,653",2
2,Dec 17,9,"$5,324,393",-34.4%,2822,-15,"$1,887","$28,604,046",3
3,Dec 814,9,"$4,996,174",-6.2%,2976,+154,"$1,679","$33,600,220",4
4,Dec 1521,12,"$3,491,705",-30.1%,1491,-1485,"$2,342","$37,091,925",5
5,Dec 2228,18,"$2,387,128",-31.6%,1106,-385,"$2,158","$39,479,053",6
6,Dec 29Jan 4,25,"$828,546",-65.3%,661,-445,"$1,253","$40,307,599",7
7,Jan 511,30,"$174,226",-79.0%,268,-393,$650,"$40,481,825",8
8,Jan 1218,33,"$160,107",-8.1%,160,-108,"$1,001","$40,641,932",9
9,Jan 1925,38,"$90,681",-43.4%,132,-28,$687,"$40,732,613",10


In [29]:
transform_movie(res2017, "2017", 17)


In [30]:
transform_movie(res2016, "2016", 16)

In [31]:
transform_movie(resMarCU, "MCU", 18)